Notes:
11/30/21 method structure finished and AR 1996 regridded

In [1]:
# imports
import numpy as np
import xarray as xr
import xesmf as xe
import Nio

In [2]:
# establishing the regridder; helper files dates do not matter
livneh_file = '/cw3e/mead/projects/cwp103/scratch/ldehaan/Livneh/livneh_unsplit_precip.2021-05-02.1996.nc'
wrf_file = '/cw3e/mead/projects/cwp103/production/output/1996122500/cf/NG-GR-YQ-BD/wrfcf_d02_1996-12-25_00_00_00.nc'
regrid_in = xr.open_dataset(wrf_file)
regrid_out = xr.open_dataset(livneh_file)
regridder = xe.Regridder(regrid_in, regrid_out, 'bilinear')

In [6]:
# regridding function creates a new .nc file for each day
def regrid(case, run, dates_lst, dates_next_day):
    
    # helper function accumulates precip in single day
    def daily_precip(day, next_day):    # input format: yyyy-mm-dd
        hours = ['01','02','03','04','05','06','07','08','09','10','11',
                 '12','13','14','15','16','17','18','19','20','21','22','23']
        added = []
        for i in hours:
            file = xr.open_dataset('/cw3e/mead/projects/cwp103/production/output/'+case+'/cf/'+run+'/wrfcf_d02_'+day+'_'+i+'_00_00.nc')
            if i == '01':
                added = file['precip_bkt'].data
                continue
            added = added + file['precip_bkt'].data
        last_hr = xr.open_dataset('/cw3e/mead/projects/cwp103/production/output/'+case+'/cf/'+run+'/wrfcf_d02_'+next_day+'_'+i+'_00_00.nc')
        added = added + last_hr['precip_bkt'].data
        return added
    
    for i in np.arange(len(dates_lst)):
        result = daily_precip(dates_lst[i], dates_next_day[i])
        wrf = xr.open_dataset('/cw3e/mead/projects/cwp103/production/output/1996122500/cf/NG-GR-YQ-BD/wrfcf_d02_1996-12-25_01_00_00.nc')
        wrf['cumulative'] = wrf['precip_bkt']
        wrf['cumulative'].data[0] = result
        dr_out = regridder(wrf['cumulative'])
        dr_out[0].to_netcdf('./wrf_'+case+'_'+run+'_precip_regrid_'+dates_lst[i]+'.nc')
        print(dates_lst[i]+' file completed.')

In [4]:
# AR 1996
dates_19 = ['1996-12-25','1996-12-26','1996-12-27','1996-12-28','1996-12-29','1996-12-30',
         '1996-12-31','1997-01-01','1997-01-02','1997-01-03','1997-01-04']
next_dates_19 = ['1996-12-26','1996-12-27','1996-12-28','1996-12-29','1996-12-30',
              '1996-12-31','1997-01-01','1997-01-02','1997-01-03','1997-01-04','1997-01-05']

lst = ['NG-GR-YQ-BD','NG-SP-NQ-PO','OO-6D','OO-CO','OO-SC','PH-MO-GF-MY','PH-MO-TK-MY',
      'PH-TH-GF-MY','PH-TH-TK-MY','VL-100','VL-120','NG-GR-NQ-PO','NG-GR-YQ-PO','NG-SP-YQ-PO','OO-CC',
      'NG-GR-NQ-BD','OO-NP','OO-VC','PH-MO-GF-YU','PH-MO-TK-YU','PH-TH-GF-YU','PH-TH-TK-YU','VL-110','VL-130',
       'CC-TA-GF-MY','NC-TH-GF-MY','CC-TH-GF-MY']

lst_19 = ['NG-GR-YQ-BD','NG-SP-NQ-PO','OO-6D','OO-CO','OO-SC','PH-MO-GF-MY','PH-MO-TK-MY',
      'PH-TH-GF-MY','PH-TH-TK-MY','VL-100','VL-120','NG-GR-NQ-PO','NG-GR-YQ-PO','NG-SP-YQ-PO','OO-CC',
      'NG-GR-NQ-BD','OO-NP','OO-VC','PH-MO-GF-YU','PH-MO-TK-YU','PH-TH-GF-YU','PH-TH-TK-YU','VL-110','VL-130',
       'NC-TH-GF-MY','CC-TH-GF-MY']

for i in lst_19:
    regrid('1996122500', i, dates_19, next_dates_19) #CC-TA missing files

/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


1996-12-25 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


1996-12-26 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


...

/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


1997-01-02 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


1997-01-03 file completed.
1997-01-04 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


In [7]:
# AR 2017
dates_17 = ['2017-02-05','2017-02-06','2017-02-07','2017-02-08','2017-02-09',
            '2017-02-10','2017-02-11','2017-02-12','2017-02-13','2017-02-14','2017-02-15']
next_dates_17 = ['2017-02-06','2017-02-07','2017-02-08','2017-02-09','2017-02-10',
                 '2017-02-11','2017-02-12','2017-02-13','2017-02-14','2017-02-15','2017-02-16']
for i in lst:
    regrid('2017020500', i, dates_17, next_dates_17)

/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2017-02-05 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2017-02-06 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


...

2017-02-13 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2017-02-14 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2017-02-15 file completed.


In [8]:
# 2010 Cloud
dates_10 = ['2010-07-07','2010-07-08','2010-07-09','2010-07-10','2010-07-11',
            '2010-07-12','2010-07-13','2010-07-14','2010-07-15','2010-07-16','2010-07-17']
next_dates_10 = ['2010-07-08','2010-07-09','2010-07-10','2010-07-11','2010-07-12',
                 '2010-07-13','2010-07-14','2010-07-15','2010-07-16','2010-07-17','2010-07-18']
for i in lst:
    regrid('2010070700', i, dates_10, next_dates_10)

/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2010-07-07 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2010-07-08 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


...

2010-07-15 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2010-07-16 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2010-07-17 file completed.


In [10]:
# 2006 Heat
dates_06 = ['2006-07-20','2006-07-21','2006-07-22','2006-07-23','2006-07-24',
            '2006-07-25','2006-07-26','2006-07-27','2006-07-28','2006-07-29','2006-07-30']
next_dates_06 = ['2006-07-21','2006-07-22','2006-07-23','2006-07-24','2006-07-25',
                 '2006-07-26','2006-07-27','2006-07-28','2006-07-29','2006-07-30','2006-07-31']
for i in lst:
    regrid('2006072000', i, dates_06, next_dates_06)

/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2006-07-20 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2006-07-21 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


...

/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2006-07-28 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2006-07-29 file completed.
2006-07-30 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


In [11]:
# 2007 Wind
dates_07 = ['2007-10-18','2007-10-19','2007-10-20','2007-10-21','2007-10-22',
            '2007-10-23','2007-10-24','2007-10-25','2007-10-26','2007-10-27','2007-10-28']
next_dates_07 = ['2007-10-19','2007-10-20','2007-10-21','2007-10-22','2007-10-23',
                 '2007-10-24','2007-10-25','2007-10-26','2007-10-27','2007-10-28','2007-10-29']
for i in lst:
    regrid('2007101800', i, dates_07, next_dates_07)

/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2007-10-18 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2007-10-19 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


...

/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2007-10-26 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


2007-10-27 file completed.
2007-10-28 file completed.


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(
